In [19]:
def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

fancy_func(1, 2, 3, 4)

10

In [20]:

def add_str():
    return '''
def add(a, b):
    return a + b
'''

def fancy_func_str():
    return '''
def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g
'''

def evoke_str():
    return add_str() + fancy_func_str() + '''
print(fancy_func(1, 2, 3, 4))
'''

prog = evoke_str()
print(prog)
y = compile(prog, '', 'exec')
exec(y)


def add(a, b):
    return a + b

def fancy_func(a, b, c, d):
    e = add(a, b)
    f = add(c, d)
    g = add(e, f)
    return g

print(fancy_func(1, 2, 3, 4))

10


In [21]:
from mxnet import nd, sym
from mxnet.gluon import nn
import time

def get_net():
    net = nn.HybridSequential()  # 这里创建HybridSequential实例
    net.add(nn.Dense(256, activation='relu'),
            nn.Dense(128, activation='relu'),
            nn.Dense(2))
    net.initialize()
    return net

x = nd.random.normal(shape=(1, 512))
net = get_net()
net(x)


[[0.07351309 0.01375049]]
<NDArray 1x2 @cpu(0)>

In [22]:
net.hybridize()
net(x)


[[0.07351309 0.01375049]]
<NDArray 1x2 @cpu(0)>

In [23]:
def benchmark(net, x):
    start = time.time()
    for i in range(1000):
        _ = net(x)
    nd.waitall()  # 等待所有计算完成方便计时
    return time.time() - start

net = get_net()
print('before hybridizing: %.4f sec' % (benchmark(net, x)))
net.hybridize()
print('after hybridizing: %.4f sec' % (benchmark(net, x)))

before hybridizing: 0.2025 sec
after hybridizing: 0.1416 sec


In [24]:
net.export('my_mlp') 、
# .json和.params文件分别为符号式程序和模型参数
# 可以通过export函数将符号式程序和模型参数保存到硬盘。

In [25]:
x = sym.var('data')
net(x) # 给它输入一个Symbol类型的变量，net(x)会返回Symbol类型的结果。

<Symbol dense15_fwd>

In [44]:
class HybridNet(nn.HybridBlock):
    def __init__(self, **kwargs):
        super(HybridNet, self).__init__(**kwargs)
        self.hidden = nn.Dense(10)
        self.output = nn.Dense(2)

    # 是hybrid_forward而不是forward,且需要F
    def hybrid_forward(self, F, x):
        print('F: ', F)
        print('x: ', x)
        x = F.relu(self.hidden(x))
        print('hidden: ', x)
        for i in range(20):
            print(i)
        return self.output(x)

In [45]:
net = HybridNet()
net.initialize()
x = nd.random.normal(shape=(1, 4))
net(x)

F:  <module 'mxnet.ndarray' from 'd:\\miniconda3\\envs\\gluon\\lib\\site-packages\\mxnet\\ndarray\\__init__.py'>
x:  
[[ 2.526586   -1.1727864  -1.7029836  -0.42165664]]
<NDArray 1x4 @cpu(0)>
hidden:  
[[0.         0.         0.19199954 0.08029822 0.         0.02240866
  0.01227654 0.24717371 0.         0.        ]]
<NDArray 1x10 @cpu(0)>
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19



[[0.03423059 0.00203426]]
<NDArray 1x2 @cpu(0)>

In [46]:
net(x)

F:  <module 'mxnet.ndarray' from 'd:\\miniconda3\\envs\\gluon\\lib\\site-packages\\mxnet\\ndarray\\__init__.py'>
x:  
[[ 2.526586   -1.1727864  -1.7029836  -0.42165664]]
<NDArray 1x4 @cpu(0)>
hidden:  
[[0.         0.         0.19199954 0.08029822 0.         0.02240866
  0.01227654 0.24717371 0.         0.        ]]
<NDArray 1x10 @cpu(0)>
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19



[[0.03423059 0.00203426]]
<NDArray 1x2 @cpu(0)>

In [47]:
net.hybridize()
net(x)
# 在hybrid_forward函数里，相同输入和中间输出全部变成了Symbol类型。

F:  <module 'mxnet.symbol' from 'd:\\miniconda3\\envs\\gluon\\lib\\site-packages\\mxnet\\symbol\\__init__.py'>
x:  <Symbol data>
hidden:  <Symbol hybridnet5_relu0>
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19



[[0.03423059 0.00203426]]
<NDArray 1x2 @cpu(0)>

In [48]:
net(x) 
# 由于hybirdize()不再打印输出。
# 对于原地操作a += b和a[:] = a + b（需改写为a = a + b）


[[0.03423059 0.00203426]]
<NDArray 1x2 @cpu(0)>